In [1]:
import torch
import torch.nn as nn
import tiktoken

C:\Users\radcl\AppData\Roaming\Python\Python313\site-packages\torch\_subclasses\functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
from gpt_model import *

In [3]:
# to make training faster on a laptop, change context_length as shown:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 256,  # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [4]:
tokenizer = tiktoken.get_encoding("gpt2")

In [5]:
model = GPTModel(GPT_CONFIG_124M)

In [6]:
start_context = "Every effort moves you"

In [7]:
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you Northwesternmailsileaks DN cottonTrend Lanka tell lootingrozen


In [8]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you forward",
                        [1107,  588, 11311]]) #  " really like chocolate"]

In [9]:
with torch.no_grad():
    logits = model(inputs)

probs = torch.softmax(logits, dim=-1) # Probability of each token in vocabulary
print(probs.shape) # Shape: (batch_size, num_tokens, vocab_size)

torch.Size([2, 3, 50257])


In [10]:
probs

tensor([[[1.1983e-05, 7.5545e-06, 5.4433e-05,  ..., 4.1403e-05,
          1.2172e-05, 1.1706e-05],
         [3.7628e-05, 5.2215e-05, 5.6870e-06,  ..., 2.1619e-05,
          3.2124e-05, 1.9698e-05],
         [2.5439e-05, 2.5208e-05, 2.4174e-05,  ..., 1.6928e-05,
          2.5453e-05, 3.1884e-05]],

        [[9.4601e-06, 1.5301e-05, 2.4985e-05,  ..., 2.7139e-05,
          1.6030e-05, 2.4479e-05],
         [1.5459e-05, 1.6101e-05, 6.5197e-06,  ..., 6.9228e-06,
          1.3582e-05, 4.4215e-05],
         [1.2005e-05, 2.6283e-05, 1.2917e-05,  ..., 1.2906e-05,
          1.5601e-05, 1.2366e-05]]])

In [11]:
# predicted tokens:
token_ids = torch.argmax(probs, dim=-1, keepdim=True)
print("Token IDs:\n", token_ids)

Token IDs:
 tensor([[[37805],
         [ 3452],
         [49313]],

        [[31304],
         [ 7878],
         [20402]]])


In [12]:
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

Targets batch 1:  effort moves you
Outputs batch 1: frames latest overpowered


In [13]:
print(f"Targets batch 2: {token_ids_to_text(targets[1], tokenizer)}")
print(f"Outputs batch 2: {token_ids_to_text(token_ids[1].flatten(), tokenizer)}")

Targets batch 2:  really like chocolate
Outputs batch 2: historicesselqa


In [14]:
targets

tensor([[ 3626,  6100,   345],
        [ 1107,   588, 11311]])

In [15]:
text_idx = 0
target_probs_1 = probs[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 1:", target_probs_1)

text_idx = 1
target_probs_2 = probs[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 2:", target_probs_2)

Text 1: tensor([1.0806e-05, 1.0220e-05, 1.8036e-05])
Text 2: tensor([1.3349e-05, 5.8420e-05, 1.0352e-05])


In [16]:
torch.set_printoptions( sci_mode=False )

In [17]:
# Compute logarithm of all token probabilities
log_probs = torch.log(torch.cat((target_probs_1, target_probs_2)))
print(log_probs)

tensor([-11.4354, -11.4912, -10.9231, -11.2240,  -9.7478, -11.4783])


In [18]:
avg_log_probs = torch.mean(log_probs)
print(avg_log_probs)
neg_avg_log_probs = avg_log_probs * -1
print(neg_avg_log_probs)

tensor(-11.0500)
tensor(11.0500)


In [19]:
logits_flat = logits.flatten(0, 1)
targets_flat = targets.flatten()

print("Flattened logits:", logits_flat.shape)
print("Flattened targets:", targets_flat.shape)

Flattened logits: torch.Size([6, 50257])
Flattened targets: torch.Size([6])


In [20]:
print(logits_flat)
print(targets_flat)

tensor([[-0.3409, -0.8022,  1.1726,  ...,  0.8990, -0.3253, -0.3643],
        [ 0.8052,  1.1328, -1.0844,  ...,  0.2510,  0.6471,  0.1580],
        [ 0.4087,  0.3996,  0.3577,  ...,  0.0013,  0.4093,  0.6345],
        [-0.5741, -0.0932,  0.3971,  ...,  0.4798, -0.0467,  0.3766],
        [-0.0861, -0.0455, -0.9495,  ..., -0.8895, -0.2156,  0.9647],
        [-0.3409,  0.4427, -0.2678,  ..., -0.2686, -0.0789, -0.3113]])
tensor([ 3626,  6100,   345,  1107,   588, 11311])


In [21]:
loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
print(loss)

tensor(11.0500)


In [22]:
# a more interpretable version of cross entropy --
# this is basically the number of tokens the model considers for predicted output
perplexity = torch.exp(loss)
print(perplexity)

tensor(62943.0078)


In [23]:
# use the short story from before for training:
with open( "humphreys.txt", "r" ) as f:
    text_data = f.read()
print(text_data[:100])

FileNotFoundError: [Errno 2] No such file or directory: 'humphreys.txt'

In [ ]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [ ]:
# separate text into training and validation sets:
# Train/validation ratio
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [ ]:
train_tokens = 0
for input_batch, target_batch in train_loader:
    train_tokens += input_batch.numel()

val_tokens = 0
for input_batch, target_batch in val_loader:
    val_tokens += input_batch.numel()

print("Training tokens:", train_tokens)
print("Validation tokens:", val_tokens)
print("All tokens:", train_tokens + val_tokens)

Training tokens: 12800
Validation tokens: 1280
All tokens: 14080


In [ ]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss


def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    # Use PyTorch 2.9 or newer for stable mps results
    major, minor = map(int, torch.__version__.split(".")[:2])
    if (major, minor) >= (2, 9):
        device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using {device} device.")


model.to(device) # no assignment model = model.to(device) necessary for nn.Module classes

torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader

with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
    train_loss = calc_loss_loader(train_loader, model, device)
    val_loss = calc_loss_loader(val_loader, model, device)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

Using cpu device.
Training loss: 10.970036811828614
Validation loss: 10.978055000305176
